# Realized Volatility Forecast

In [224]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor

## Exploratory Data Analysis

In [225]:
spyTicker = yf.Ticker("SPY") # SPDR S&P 500 ETF Trust
vixTicker = yf.Ticker("^VIX") # VIX
df_spy = spyTicker.history(period="max")
df_vix = vixTicker.history(period="max")

In [226]:
# Remove extra rows from vix to match spy
df_vix = df_vix.drop(df_vix.loc['1990':'1993-01-28'].index)

In [227]:
#Remove Timestamps to make rows line up better across datasets
df_spy.index = df_spy.index.date
df_vix.index = df_vix.index.date

In [228]:
df_spy

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
1993-01-29,24.330324,24.330324,24.209278,24.313032,1003200,0.0,0.0,0.0
1993-02-01,24.330316,24.485947,24.330316,24.485947,480500,0.0,0.0,0.0
1993-02-02,24.468671,24.555132,24.416793,24.537840,201300,0.0,0.0,0.0
1993-02-03,24.572427,24.814520,24.555135,24.797228,529400,0.0,0.0,0.0
1993-02-04,24.883686,24.952855,24.607008,24.900978,531500,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2025-11-13,680.500000,680.859985,670.520020,672.039978,103457800,0.0,0.0,0.0
2025-11-14,665.380005,675.659973,663.270020,671.929993,96846700,0.0,0.0,0.0
2025-11-17,669.700012,673.710022,662.169983,665.669983,90456100,0.0,0.0,0.0
2025-11-18,662.099976,665.119995,655.859985,660.080017,114467500,0.0,0.0,0.0


In [229]:
df_vix

,Open,High,Low,Close,Volume,Dividends,Stock Splits
1993-01-29,12.490000,13.160000,12.420000,12.420000,0,0.0,0.0
1993-02-01,12.510000,12.920000,12.180000,12.330000,0,0.0,0.0
1993-02-02,12.470000,12.890000,12.220000,12.250000,0,0.0,0.0
1993-02-03,11.980000,12.340000,11.790000,12.120000,0,0.0,0.0
1993-02-04,11.860000,12.840000,11.690000,12.290000,0,0.0,0.0
...,...,...,...,...,...,...,...
2025-11-14,21.330000,23.030001,19.559999,19.830000,0,0.0,0.0
2025-11-17,19.580000,23.440001,19.540001,22.379999,0,0.0,0.0
2025-11-18,23.540001,25.840000,22.910000,24.690001,0,0.0,0.0
2025-11-19,24.520000,24.790001,21.930000,23.660000,0,0.0,0.0


In [230]:
#join datasets
dataset = pd.merge(df_spy, df_vix, left_index=True, right_index=True, suffixes=('_SPY','_VIX'))
dataset = dataset.drop(['Dividends_SPY','Stock Splits_SPY', 'Capital Gains', 'Dividends_VIX', 'Stock Splits_VIX', 'Volume_VIX'], axis=1)

# Load FFER data, make Column A the index
FFER = pd.read_csv("FFER.csv", index_col=0, parse_dates=True)

# Rename FFER column 
FFER.columns = ["FFER"]

# Merge on the index
dataset = dataset.join(FFER, how="left")


# Load US Economic Uncertainty data, make Column A the index
econ_uncert = pd.read_csv("FFER.csv", index_col=0, parse_dates=True)

# Rename US Econmic Unceratainty column 
econ_uncert.columns = ["Economic Uncertainty"]

# Merge on the index
dataset = dataset.join(econ_uncert, how="left")

print(dataset.head())
print(dataset.isna().sum())


             Open_SPY   High_SPY    Low_SPY  Close_SPY  Volume_SPY  Open_VIX  \
1993-01-29  24.330324  24.330324  24.209278  24.313032     1003200     12.49   
1993-02-01  24.330316  24.485947  24.330316  24.485947      480500     12.51   
1993-02-02  24.468671  24.555132  24.416793  24.537840      201300     12.47   
1993-02-03  24.572427  24.814520  24.555135  24.797228      529400     11.98   
1993-02-04  24.883686  24.952855  24.607008  24.900978      531500     11.86   

            High_VIX  Low_VIX  Close_VIX  FFER  Economic Uncertainty  
1993-01-29     13.16    12.42      12.42  3.02                  3.02  
1993-02-01     12.92    12.18      12.33  3.25                  3.25  
1993-02-02     12.89    12.22      12.25  3.18                  3.18  
1993-02-03     12.34    11.79      12.12  3.56                  3.56  
1993-02-04     12.84    11.69      12.29  3.08                  3.08  
Open_SPY                0
High_SPY                0
Low_SPY                 0
Close_SPY      

In [231]:
# Download 3M T-Bill yield
tbill = yf.Ticker("^IRX")
df_tbill = tbill.history(period="max")

# Align date index
df_tbill.index = df_tbill.index.date
df_tbill = df_tbill[['Close']]  # Take yield

# Rename column
df_tbill.columns = ['TBill_3M']

# Merge with your main dataset
dataset = dataset.join(df_tbill, how="left")

In [232]:
dataset

,Open_SPY,High_SPY,Low_SPY,Close_SPY,Volume_SPY,Open_VIX,High_VIX,Low_VIX,Close_VIX,FFER,Economic Uncertainty,TBill_3M
1993-01-29,24.330324,24.330324,24.209278,24.313032,1003200,12.490000,13.160000,12.420000,12.420000,3.02,3.02,2.900
1993-02-01,24.330316,24.485947,24.330316,24.485947,480500,12.510000,12.920000,12.180000,12.330000,3.25,3.25,2.900
1993-02-02,24.468671,24.555132,24.416793,24.537840,201300,12.470000,12.890000,12.220000,12.250000,3.18,3.18,2.960
1993-02-03,24.572427,24.814520,24.555135,24.797228,529400,11.980000,12.340000,11.790000,12.120000,3.56,3.56,2.930
1993-02-04,24.883686,24.952855,24.607008,24.900978,531500,11.860000,12.840000,11.690000,12.290000,3.08,3.08,2.900
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-11-13,680.500000,680.859985,670.520020,672.039978,103457800,17.610001,21.309999,17.510000,20.000000,3.88,3.88,3.793
2025-11-14,665.380005,675.659973,663.270020,671.929993,96846700,21.330000,23.030001,19.559999,19.830000,3.88,3.88,3.788
2025-11-17,669.700012,673.710022,662.169983,665.669983,90456100,19.580000,23.440001,19.540001,22.379999,3.88,3.88,3.772
2025-11-18,662.099976,665.119995,655.859985,660.080017,114467500,23.540001,25.840000,22.910000,24.690001,NaN,NaN,3.772


In [233]:
# SPY features
# daily log price and  natural log-returns for SPY
dataset["log_close_SPY"] = np.log(dataset["Close_SPY"])
dataset["log_open_SPY"] = np.log(dataset["Open_SPY"])
dataset["ret_1d"] = dataset["log_close_SPY"].diff(1)   # daily natural log-return

# intraday turbulance
dataset["intraday_OC_abs"] = np.abs(dataset["log_open_SPY"] - dataset["log_close_SPY"])
dataset["intraday_OC"] = dataset["log_open_SPY"] - dataset["log_close_SPY"]
dataset["intraday_CO_abs"] = np.abs(dataset["log_close_SPY"].shift(1) - dataset["log_open_SPY"]) #What happend overnight? price jumped/Crashed? 
dataset["intraday_CH"] = dataset["log_close_SPY"].shift(1) - np.log(dataset["High_SPY"])
dataset["intraday_CL"] = dataset["log_close_SPY"].shift(1) - np.log(dataset["Low_SPY"])
dataset["intraday_CO"] = dataset["log_close_SPY"].shift(1) - dataset["log_open_SPY"]
dataset["intraday_HL"] = np.log(dataset["High_SPY"]) - np.log(dataset["Low_SPY"])
dataset["intraday_HL_abs"] = np.abs(np.log(dataset["High_SPY"]) - np.log(dataset["Low_SPY"]))

# lagged returns (1 and 2 days)
dataset["ret_lag1"] = dataset["ret_1d"].shift(1)
dataset["ret_lag2"] = dataset["ret_1d"].shift(2)
dataset["intraday_HL_abs_lag1"] = dataset["intraday_HL_abs"].shift(1)
dataset["intraday_HL_abs_lag2"] = dataset["intraday_HL_abs"].shift(2)
dataset["intraday_HL_lag1"] = dataset["intraday_HL"].shift(1)
dataset["intraday_HL_lag2"] = dataset["intraday_HL"].shift(2)

# rolling statistics of returns
dataset["ret_roll_mean_5d"] = dataset["ret_1d"].rolling(5, min_periods=1).mean()
dataset["ret_roll_std_5d"] = dataset["ret_1d"].rolling(5, min_periods=1).std()
dataset["ret_roll_abs_5d"] = np.abs(dataset["ret_1d"].rolling(5, min_periods=1).sum())

dataset["ret_roll_mean_22d"] = dataset["ret_1d"].rolling(22, min_periods=1).mean()
dataset["ret_roll_std_22d"] = dataset["ret_1d"].rolling(22, min_periods=1).std()
dataset["ret_roll_abs_22d"] = np.abs(dataset["ret_1d"].rolling(22, min_periods=1).sum())

dataset["intraday_roll_CO_mean_5d"] = dataset["intraday_CO"].rolling(5, min_periods=1).mean()
dataset["intraday_roll_CO_std_5d"] = dataset["intraday_CO"].rolling(5,min_periods=1).std()
dataset["intraday_roll_CO_mean_22d"] = dataset["intraday_CO"].rolling(22,min_periods=1).mean()
dataset["intraday_roll_CO_std_22d"] = dataset["intraday_CO"].rolling(22,min_periods=1).std()


# Technical indicators: moving averages on Close (5, 10, 22 days)
dataset["ma_5"] = dataset["log_close_SPY"].rolling(window=5, min_periods=1).mean()
dataset["ma_10"] = dataset["log_close_SPY"].rolling(window=10,min_periods=1).mean()
dataset["ma_22"] = dataset["log_close_SPY"].rolling(window=22,min_periods=1).mean()

dataset["momentum_5d"] = dataset["log_close_SPY"] - dataset["log_close_SPY"].shift(5)
dataset["momentum_22d"] = dataset["log_close_SPY"] - dataset["log_close_SPY"].shift(22)

#Relative Strength Index. Measure between 0 - 100 indicating if more gains or loss has occured in predefined time period
delta = dataset["ret_1d"]

gain = delta.clip(lower=0)  
loss = -delta.clip(upper=0) 

avg_gain = gain.rolling(5, min_periods=1).mean()
avg_loss = loss.rolling(5, min_periods=1).mean()

# Avoid divide-by-zero
avg_loss = avg_loss.replace(0, 1e-10)

RS = avg_gain / avg_loss
RSI = 100 - (100 / (1 + RS))

dataset["RSI_5d"] = RSI

# Bollinger Bands. Measure how close or far is the current price from the moving average of a predefine time period
dataset["BB_low"] = dataset["ma_22"] - (2 * dataset["log_close_SPY"].rolling(22).std())
dataset["BB_high"] = dataset["ma_22"] - (2 * dataset["log_close_SPY"].rolling(22).std())

dataset["BB_percentage_diff"] =  (dataset["BB_high"] - dataset["BB_low"]) / dataset["ma_22"]


dataset["BB_price_location"] =  (dataset["log_close_SPY"] - dataset["BB_low"]) / (dataset["BB_high"] - dataset["BB_low"] + 1e-10)


#Average True Range. Pick the largets movement between H/L, Ct-1/H, Ct-1/L
dataset["TR"] = dataset[["intraday_HL","intraday_CH", "intraday_CL"]].max(axis=1)
dataset["ATR"] = dataset["TR"].rolling(14, min_periods=1).mean()

# Rate of change (ROC) – 5 and 10 day
dataset["roc_5"] = dataset["log_close_SPY"].pct_change(periods=5)
dataset["roc_10"] = dataset["log_close_SPY"].pct_change(periods=10)

# Volume-related feature (log-volume + 10-day rolling mean)
dataset["log_volume"] = np.log(dataset["Volume_SPY"].replace(0, np.nan))
dataset["vol_roll_mean_10"] = dataset["log_volume"].rolling(window=10,min_periods=1).mean()

# Realized variance and lagged relalized variance daily, weekly, monthly
dataset["rvar_1d"] = dataset["ret_1d"] ** 2
dataset["rvar_5d"] = dataset["rvar_1d"].rolling(window=5,min_periods=1).sum()
dataset["rvar_22d"] = dataset["rvar_1d"].rolling(window=22,min_periods=1).sum()

# Realized volatility and lagged relalized volatility daily, weekly, monthly
dataset["rvol_1d"] = np.sqrt(dataset["rvar_1d"]) # Y lable
dataset["rvol_5d"] = np.sqrt(dataset["rvar_5d"])
dataset["rvol_22d"] = np.sqrt(dataset["rvar_22d"])

In [234]:
dataset = dataset.dropna()

In [235]:
dataset

,Open_SPY,High_SPY,Low_SPY,Close_SPY,Volume_SPY,Open_VIX,High_VIX,Low_VIX,Close_VIX,FFER,...,roc_5,roc_10,log_volume,vol_roll_mean_10,rvar_1d,rvar_5d,rvar_22d,rvol_1d,rvol_5d,rvol_22d
1993-03-03,24.900963,24.987425,24.866379,24.970133,280100,13.140000,13.710000,12.930000,13.130000,3.29,...,0.006123,0.011987,12.542902,11.702639,1.732942e-05,0.000248,0.001409,0.004163,0.015754,0.037531
1993-03-04,25.004743,25.004743,24.831820,24.831820,89500,12.520000,14.020000,12.500000,13.440000,3.10,...,0.003721,0.010469,11.401994,11.558547,3.085300e-05,0.000275,0.001389,0.005555,0.016570,0.037272
1993-03-05,24.866392,24.970146,24.745346,24.762638,40000,13.200000,14.130000,12.980000,14.080000,3.02,...,0.002409,0.008451,10.596635,11.572186,7.783474e-06,0.000280,0.001392,0.002790,0.016744,0.037316
1993-03-08,24.814505,25.315983,24.814505,25.315983,50800,15.000000,16.299999,14.950000,16.219999,3.06,...,0.010201,0.014254,10.835652,11.340831,4.884073e-04,0.000761,0.001770,0.022100,0.027583,0.042075
1993-03-09,25.264125,25.281417,25.177663,25.229540,169300,14.020000,15.430000,13.780000,14.170000,2.98,...,0.004512,0.013408,12.039428,11.261653,1.169916e-05,0.000556,0.001765,0.003420,0.023581,0.042007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-11-11,679.950012,683.570007,678.729980,683.000000,58953400,17.900000,18.010000,17.250000,17.280001,3.87,...,0.001754,-0.000907,17.892258,18.158817,5.228767e-06,0.000374,0.001365,0.002287,0.019346,0.036946
2025-11-12,684.789978,684.960022,680.950012,683.380005,62312500,17.209999,18.059999,17.100000,17.510000,3.87,...,0.001308,-0.000896,17.947673,18.126998,3.093822e-07,0.000363,0.001133,0.000556,0.019043,0.033667
2025-11-13,680.500000,680.859985,670.520020,672.039978,103457800,17.610001,21.309999,17.510000,20.000000,3.88,...,0.000396,-0.001767,18.454674,18.157400,2.800017e-04,0.000526,0.001412,0.016733,0.022940,0.037576
2025-11-14,665.380005,675.659973,663.270020,671.929993,96846700,21.330000,23.030001,19.559999,19.830000,3.88,...,0.000220,-0.002293,18.388640,18.167933,2.678863e-08,0.000525,0.001392,0.000164,0.022920,0.037314


# Preprocessing

In [236]:
X = dataset
y = dataset["rvol_1d"].shift(-1)

print("shape of x:", X.shape)
print("shape of y:", y.tail())
X = X.iloc[:-1]
y = y.iloc[:-1]


shape of x: (8215, 61)
shape of y: 2025-11-11    0.000556
2025-11-12    0.016733
2025-11-13    0.000164
2025-11-14    0.009360
2025-11-17         NaN
Name: rvol_1d, dtype: float64


In [237]:
print("shape of x:", X.shape)
print("shape of y:", y.shape)
print("last 5 rows of y:", y.tail())

shape of x: (8214, 61)
shape of y: (8214,)
last 5 rows of y: 2025-11-10    0.002287
2025-11-11    0.000556
2025-11-12    0.016733
2025-11-13    0.000164
2025-11-14    0.009360
Name: rvol_1d, dtype: float64


In [238]:
#Split data between train, validate, test (60, 20, 20)
X_tmp, X_test, y_tmp, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

X_train, X_val, y_train, y_val = train_test_split(X_tmp, y_tmp, test_size=0.25, shuffle=False)

# Random Forest Model

In [239]:
# Initialize model
rf = RandomForestRegressor(
    n_estimators=1000, 
    max_depth=8,    
    random_state=42
)

# Fit on training data
rf.fit(X_train, y_train)

# Predictions
y_train_pred = rf.predict(X_train)
y_test_pred  = rf.predict(X_test)

# Training metrics 
train_mse_rf  = mean_squared_error(y_train, y_train_pred)
train_rmse_rf = train_mse_rf ** 0.5
train_mae_rf  = mean_absolute_error(y_train, y_train_pred)
train_r2_rf   = r2_score(y_train, y_train_pred)

# Testing metrics
test_mse_rf = mean_squared_error(y_test, y_test_pred)
test_rmse_rf = test_mse_rf**0.5
test_mae_rf = mean_absolute_error(y_test, y_test_pred)
test_r2_rf = rf.score(X_test, y_test)


# Print comparison 
print("Training Metrics")
print("RMSE (train):", train_rmse_rf)
print("MAE  (train):", train_mae_rf)
print("R^2  (train):", train_r2_rf)

print("\nTest Metrics")
print("RMSE:", test_rmse_rf)
print("MAE:", test_mae_rf)
print("R^2: ", test_r2_rf)

Training Metrics
RMSE (train): 0.00583547119191665
MAE  (train): 0.004402120929468363
R^2  (train): 0.5974000195250195

Test Metrics
RMSE: 0.008138683190884378
MAE: 0.005353344312141006
R^2:  0.2921577502136855


## Most Representative Features

In [240]:
fi = pd.DataFrame({
    "feature": X_train.columns,
    "importance": rf.feature_importances_
}).sort_values("importance", ascending=False)

print(fi)

               feature  importance
6             High_VIX    0.199023
8            Close_VIX    0.080360
5             Open_VIX    0.062699
52              roc_10    0.057175
50                 ATR    0.045307
..                 ...         ...
1             High_SPY    0.002346
3            Close_SPY    0.002160
0             Open_SPY    0.002119
13        log_open_SPY    0.001875
47  BB_percentage_diff    0.000000

[61 rows x 2 columns]


# Linear Regression 

In [241]:
#Standardize data with 0 mean and variance
std = StandardScaler().fit(X_train)
X_train_std = std.transform(X_train)
X_val_std = std.transform(X_val)
X_test_std = std.transform(X_test)


lr = LinearRegression()
lr.fit(X_train_std, y_train)

# Predictions
y_train_pred = lr.predict(X_train)
y_test_pred  = lr.predict(X_test)

# Training metrics 
train_mse_lr = mean_squared_error(y_train, y_train_pred)
train_rmse_lr = train_mse_lr ** 0.5
train_mae_lr  = mean_absolute_error(y_train, y_train_pred)
train_r2_lr   = r2_score(y_train, y_train_pred)

# Test metrics
test_mse_lr  = mean_squared_error(y_test, y_test_pred)
test_rmse_lr = test_mse_lr ** 0.5
test_mae_lr  = mean_absolute_error(y_test, y_test_pred)
test_r2_lr   = r2_score(y_test, y_test_pred)

# Print comparison 
print("Training Metrics")
print("RMSE (train):", train_rmse_lr)
print("MAE  (train):", train_mae_lr)
print("R^2  (train):", train_r2_lr)

print("\nTest Metrics")
print("RMSE (test):", test_rmse_lr)
print("MAE  (test):", test_mae_lr)
print("R^2  (test):", test_r2_lr)


Training Metrics
RMSE (train): 70860.4672351699
MAE  (train): 52847.19101900135
R^2  (train): -59364923696497.27

Test Metrics
RMSE (test): 66317.0961825983
MAE  (test): 51109.481215342035
R^2  (test): -46997980615879.32


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


XGBoost

In [242]:

xgb = XGBRegressor(
    n_estimators=1500,
    learning_rate=0.02,
    max_depth=4,
    subsample=0.7,
    colsample_bytree=0.6,
    reg_alpha=0.5,
    reg_lambda=2,
    min_child_weight=5,
    objective='reg:squarederror',
    random_state=42,
)

# Fit on training data
xgb.fit(X_train, y_train)

# Predictions
y_train_pred = xgb.predict(X_train)
y_test_pred  = xgb.predict(X_test)

# Training metrics 
train_mse_xgb  = mean_squared_error(y_train, y_train_pred)
train_rmse_xgb = train_mse_xgb ** 0.5
train_mae_xgb  = mean_absolute_error(y_train, y_train_pred)
train_r2_xgb   = r2_score(y_train, y_train_pred)

# Test metrics
test_mse_xgb  = mean_squared_error(y_test, y_test_pred)
test_rmse_xgb = test_mse_xgb ** 0.5
test_mae_xgb  = mean_absolute_error(y_test, y_test_pred)
test_r2_xgb   = r2_score(y_test, y_test_pred)

# Print comparison 
print("Training Metrics")
print("RMSE (train):", train_rmse_xgb)
print("MAE  (train):", train_mae_xgb)
print("R^2  (train):", train_r2_xgb)

print("\nTest Metrics")
print("RMSE (test):", test_rmse_xgb)
print("MAE  (test):", test_mae_xgb)
print("R^2  (test):", test_r2_xgb)

# Metrics

In [243]:
cols_names = ['RMSE', 'MAE', 'R^2']
index_names = ['Random Forest', 'Linear Regression', 'XGB']

metrics = pd.DataFrame(index=index_names, columns=cols_names)
metrics.loc['Random Forest'] = [test_rmse_rf, test_mae_rf, test_r2_rf]
metrics.loc['Linear Regression'] = [test_rmse_lr, test_mae_lr, test_r2_lr]
metrics.loc['XGB'] = [test_rmse_xgb, test_mae_xgb, test_r2_xgb]
metrics

,RMSE,MAE,R^2
Random Forest,0.008139,0.005353,0.292158
Linear Regression,66317.096183,51109.481215,-46997980615879.320312
XGB,NaN,NaN,NaN
